In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/speech-based-classification-layer-9/valid.csv
/kaggle/input/speech-based-classification-layer-9/train.csv
/kaggle/input/speech-based-classification-layer-9/test.csv


In [2]:
# Imports


from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import joblib

import math
import seaborn as sns
import warnings                   # To ignore the warnings
import pandas as pd
import numpy as np          # For mathematical calculations
import matplotlib.pyplot as plt  # For plotting graphs
from datetime import datetime    # To access datetime
from pandas import Series        # To work on series

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

from xgboost import XGBClassifier

# import RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV

%matplotlib inline
warnings.filterwarnings("ignore")


### Data importing

In [3]:
train_layer_9 = pd.read_csv("/kaggle/input/speech-based-classification-layer-9/train.csv")
valid_layer_9 = pd.read_csv("/kaggle/input/speech-based-classification-layer-9/valid.csv")

Test Dataset

In [4]:
test_layer_9 = pd.read_csv("/kaggle/input/speech-based-classification-layer-9/test.csv")

In [10]:
test_layer_9.head()

,ID,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_759,feature_760,feature_761,feature_762,feature_763,feature_764,feature_765,feature_766,feature_767,feature_768
0,1,0.013112,0.130904,0.020284,0.063018,-0.034321,-0.073516,-0.030659,-0.064994,0.024153,...,0.037803,0.241121,0.079949,-0.186099,-0.096718,0.126006,-0.023069,0.190374,0.146516,0.038047
1,2,0.005934,0.138592,-0.007000,0.055925,-0.021927,-0.084788,0.013339,0.060811,-0.011344,...,-0.098701,0.042921,0.018571,-0.114785,-0.105186,0.059050,0.021443,0.013027,0.046826,-0.026682
2,3,-0.067210,0.078710,-0.044344,0.101248,-0.074331,-0.088951,0.074616,0.007231,0.021091,...,0.029569,0.053370,0.096232,-0.369018,-0.066708,-0.003730,-0.063339,-0.044497,-0.024363,-0.042594
3,4,-0.005678,0.060703,0.033954,0.068771,-0.039923,-0.186583,0.014921,0.020791,0.017441,...,-0.095406,0.018405,-0.018047,-0.080393,-0.114030,0.048255,0.033839,0.035026,-0.047988,-0.038252
4,5,-0.076360,0.061095,-0.004938,0.066692,-0.040454,-0.005141,-0.003213,0.025721,0.083558,...,-0.003129,0.045479,0.057146,-0.194466,0.000739,0.018702,0.013192,-0.038486,0.033358,-0.038452


In [5]:
x_train = train_layer_9.copy().drop(
    columns=["label_1", "label_2", "label_3", "label_4"]
)
x_valid = valid_layer_9.copy().drop(
    columns=["label_1", "label_2", "label_3", "label_4"]
)
x_feature_names = ["feature_" + str(i) for i in range(1, 769)]

id_train = train_layer_9["label_1"].to_frame()
age_train = train_layer_9["label_2"].to_frame()  # id has NaN
gender_train = train_layer_9["label_3"].to_frame()
accent_train = train_layer_9["label_4"].to_frame()  # Accent has bias to 6

id_valid = valid_layer_9["label_1"].to_frame()
age_valid = valid_layer_9["label_2"].to_frame()
gender_valid = valid_layer_9["label_3"].to_frame()
accent_valid = valid_layer_9["label_4"].to_frame()

# Scaling using RobustScaler
scaler_robust = RobustScaler()
scaler_robust.fit(x_train)

x_train_scaled = pd.DataFrame(scaler_robust.transform(x_train), columns=x_feature_names)
x_valid_scaled = pd.DataFrame(scaler_robust.transform(x_valid), columns=x_feature_names)

Test Dataset

In [11]:
x_test = test_layer_9.copy().drop(
    columns=["ID"]
)
x_test_scaled = pd.DataFrame(scaler_robust.transform(x_test), columns=x_feature_names)

### Support Functions

In [12]:
def validate_model(model, x_valid, y_valid):
    y_pred = model.predict(x_valid)
    accuracy = accuracy_score(y_valid, y_pred)
    print("Accuracy: ", accuracy)
    cm = confusion_matrix(y_valid, y_pred)
    print("Confusion matrix: ")
    print(cm)
    print("Precision, recall, f1-score: ")
    prfs_ = precision_recall_fscore_support(
        y_valid, y_pred, average="weighted")
    return prfs_, cm

def do_pca(train, valid, variance: float = None, n_components: int = None):
    pca_obj = PCA(n_components=0.95, svd_solver="full")
    if variance:
        pca_obj = PCA(n_components=variance, svd_solver="full")
    elif n_components:
        pca_obj = PCA(n_components=n_components)
    pca_obj.fit(train)
    n_components = pca_obj.components_.shape[0]

    x_train = pd.DataFrame(
        pca_obj.transform(train),
        columns=["feature_pca_" + str(i) for i in range(1, n_components + 1)],
    )
    x_valid = pd.DataFrame(
        pca_obj.transform(valid),
        columns=["feature_pca_" + str(i) for i in range(1, n_components + 1)],
    )

    return x_train, x_valid, n_components, pca_obj

def fit_and_score(classifier_models, x_train, x_valid, y_train, y_valid):
    """
    Fits and evaluates given classifier models.
    classifier_models : a dict of different Scikit-Learn classifier models
    x_train : training data (no labels)
    x_valid : validation data (no labels)
    y_train : training labels
    y_valid : validation labels
    """
    # Set random seed
    np.random.seed(42)
    # Make a dictionary to keep model scores
    classifier_model_scores = {}
    # Loop through classifier_models
    for name, model in classifier_models.items():
        print(f"Fitting {name}...")
        # Fit the model to the data
        model.fit(x_train, y_train)
        # Evaluate the model and append its score to classifier_model_scores
        classifier_model_scores[name] = model.score(x_valid, y_valid)
        print("Done fitting and scoring model.")
    return classifier_model_scores

def tune_hyperparameters(classifier_models, grid, x_train, x_valid, y_train, y_valid):
    """
    Fits and evaluates given classifier models.
    classifier_models : a dict of different Scikit-Learn classifier models
    grid : a dict of hyperparameters to tune
    x_train : training data (no labels)
    x_valid : validation data (no labels)
    y_train : training labels
    y_valid : validation labels
    """
    # Set random seed
    np.random.seed(42)
    # Make a dictionary to keep model scores
    classifier_model_scores = {}
    trained_searches = {}
    # Loop through classifier_models
    for name, model in classifier_models.items():
        # Setup random hyperparameter search for model
        rs_model = RandomizedSearchCV(
            estimator=model,
            param_distributions=grid,
            n_iter=20,
            cv=5,
            verbose=3,
            random_state=42,
            n_jobs=-1,
        )
        # Fit random hyperparameter search model
        fileName = name + "_rs_model" + ".pkl"
        rs_model.fit(x_train, y_train)
        # Export rs_model using joblib
        joblib.dump(rs_model, fileName)
        trained_searches[name] = rs_model

        # Evaluate the model and append its score to classifier_model_scores
        classifier_model_scores[name] = rs_model.score(x_valid, y_valid)
        print("classifier model scores", classifier_model_scores)
    return trained_searches

## Label 01

In [13]:
id_data_train_cat = pd.concat([x_train_scaled, id_train], axis=1)
id_data_valid_cat = pd.concat([x_valid_scaled, id_valid], axis=1)

# Remove rows with null values
id_data_cleaned_train_cat = id_data_train_cat.dropna()
id_data_cleaned_valid_cat = id_data_valid_cat.dropna()

# Separate X and y again
id_x_train_cat = id_data_cleaned_train_cat.drop(columns=["label_1"])
id_y_train_cat = id_data_cleaned_train_cat["label_1"]
id_x_valid_cat = id_data_cleaned_valid_cat.drop(columns=["label_1"])
id_y_valid_cat = id_data_cleaned_valid_cat["label_1"].to_frame()

Test Dataset

In [15]:

# Separate X and y again
id_x_test_cat = x_test_scaled

In [16]:
id_initial = SVC()
id_initial.fit(id_x_train_cat, id_y_train_cat)

SVC()

In [22]:
precision_recall_fscore_svc_initial, cm_initial = validate_model(id_initial, id_x_valid_cat, id_y_valid_cat)

print("Precision, Recall and F1 Score:", precision_recall_fscore_svc_initial)

Accuracy:  0.9466666666666667
Confusion matrix: 
[[12  0  0 ...  0  0  0]
 [ 0  9  0 ...  0  0  0]
 [ 0  0 11 ...  0  0  0]
 ...
 [ 0  0  0 ... 20  0  0]
 [ 0  0  0 ...  0 10  0]
 [ 0  0  0 ...  0  1  9]]
Precision, recall, f1-score: 
Precision, Recall and F1 Score: (0.9535477330661541, 0.9466666666666667, 0.9474071091104397, None)


In [20]:
test_predictions = id_initial.predict(id_x_test_cat)
print(test_predictions[:10])
pd.DataFrame(test_predictions, columns=["label_1"]).to_csv("id_test_predictions.csv")

[26 18 16  7 58 46  7 22 29 26]


#### PCA

In [21]:
id_x_train_pca_85, id_x_valid_pca_85, id_n_components, id_pca_cat = do_pca(
    id_x_train_cat, id_x_valid_cat, n_components=70
)
print("Number of components: ", id_n_components)

Number of components:  70


Test Dataset

In [23]:
id_x_test_pca_85 = pd.DataFrame(
    id_pca_cat.transform(id_x_test_cat),
    columns=["feature_pca_" + str(i) for i in range(1, id_n_components + 1)],
)

In [24]:
id_final = SVC()
id_final.fit(id_x_train_pca_85, id_y_train_cat)

SVC()

In [25]:
precision_recall_fscore_svc_final, cm_final = validate_model(id_final, id_x_valid_pca_85, id_y_valid_cat)

print("Precision, Recall and F1 Score:", precision_recall_fscore_svc_final)

Accuracy:  0.9053333333333333
Confusion matrix: 
[[11  0  0 ...  0  0  0]
 [ 0  9  0 ...  0  0  0]
 [ 0  0 10 ...  0  0  0]
 ...
 [ 0  0  0 ... 20  0  0]
 [ 0  0  0 ...  0 10  0]
 [ 0  0  0 ...  0  0  9]]
Precision, recall, f1-score: 
Precision, Recall and F1 Score: (0.9180403130982078, 0.9053333333333333, 0.906750374001572, None)


### Hyperparameter Tuning SVC

In [26]:
id_grid = {
    "C": [1, 10, 100, 1000],
    "gamma": [0.1, 0.01, 0.001, 0.0001],
    "kernel": ["rbf", "poly", "sigmoid"],
    "degree": [1, 2, 3, 4, 5],
}

id_rs_model = tune_hyperparameters(
    {"id_final": SVC()}, id_grid, id_x_train_pca_85, id_x_valid_pca_85, id_y_train_cat, id_y_valid_cat
)

id_precision_recall_fscore_svc_final, id_cm_final = validate_model(id_rs_model["id_final"].best_estimator_, id_x_valid_pca, id_y_valid_cat)

print(id_rs_model["id_final"].best_params_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END C=0.1, degree=1, gamma=1, kernel=rbf;, score=0.017 total time= 3.7min
[CV 5/5] END C=0.1, degree=1, gamma=1, kernel=rbf;, score=0.017 total time= 3.9min
[CV 2/5] END C=0.1, degree=2, gamma=1, kernel=rbf;, score=0.017 total time= 3.8min
[CV 1/5] END C=0.1, degree=2, gamma=0.01, kernel=rbf;, score=0.741 total time= 2.3min
[CV 5/5] END C=0.1, degree=2, gamma=0.01, kernel=rbf;, score=0.734 total time= 2.4min
[CV 4/5] END C=1, degree=1, gamma=1, kernel=rbf;, score=0.017 total time= 3.8min
[CV 2/5] END C=1, degree=1, gamma=0.01, kernel=rbf;, score=0.901 total time= 2.4min
[CV 1/5] END C=1, degree=2, gamma=1, kernel=rbf;, score=0.017 total time= 3.9min
[CV 5/5] END C=1, degree=2, gamma=1, kernel=rbf;, score=0.017 total time= 3.8min
[CV 3/5] END C=0.1, degree=1, gamma=1, kernel=rbf;, score=0.017 total time= 3.8min
[CV 2/5] END C=0.1, degree=1, gamma=0.01, kernel=rbf;, score=0.737 total time= 2.4min
[CV 1/5] END C=0.1, deg

In [ ]:
id_label_1_final_preds = id_rs_model["id_final"].best_estimator_.predict(id_x_test_pca_85)
pd.DataFrame(id_label_1_final_preds, columns=["label_1"]).to_csv("id_label_1_final_preds.csv")

# Label 2 - Age

In [ ]:
# Handling NaN values in the age
# Combine X and y into a single DataFrame

age_data_train_cat = pd.concat([x_train_scaled, age_train], axis=1)
age_data_valid_cat = pd.concat([x_valid_scaled, age_valid], axis=1)

# Remove rows with null values
age_data_cleaned_train_cat = age_data_train_cat.dropna()
age_data_cleaned_valid_cat = age_data_valid_cat.dropna()

# Separate X and y again
age_x_train_cat = age_data_cleaned_train_cat.drop(columns=["label_2"])
age_y_train_cat = age_data_cleaned_train_cat["label_2"]
age_x_valid_cat = age_data_cleaned_valid_cat.drop(columns=["label_2"])
age_y_valid_cat = age_data_cleaned_valid_cat["label_2"].to_frame()

In [ ]:
age_x_train_pca, age_x_valid_pca, age_n_components, age_pca_cat = do_pca(
    age_x_train_cat, age_x_valid_cat, n_components=180
)

In [ ]:
age_x_test_pca_85 = pd.DataFrame(
    age_pca_cat.transform(id_x_test_cat),
    columns=["feature_pca_" + str(i) for i in range(1, age_n_components + 1)],
)

In [ ]:
age_final = SVC()
age_final.fit(age_x_train_pca, age_y_train_cat)

age_precision_recall_fscore_svc_final, age_cm_final = validate_model(age_final, age_x_valid_pca, age_y_valid_cat)

print("Precision, Recall and F1 Score:", age_precision_recall_fscore_svc_final)

In [ ]:
# hyperparameter tuning for svc
age_final_models = {
    "age_SVC": SVC(),
}

age_grid = {
    "C": [1, 10, 100, 1000],
    "gamma": [0.1, 0.01, 0.001, 0.0001],
    "kernel": ["rbf", "poly", "sigmoid"],
    "degree": [1, 2, 3, 4, 5],
}

age_rs_model = tune_hyperparameters(
    age_final_models, age_grid, age_x_train_pca, age_x_valid_pca, age_y_train_cat, age_y_valid_cat
)

age_precision_recall_fscore_svc_final, age_cm_final = validate_model(age_rs_model["age_SVC"].best_estimator_, age_x_valid_pca, age_y_valid_cat)


print(age_rs_model["age_SVC"].best_params_)

In [ ]:
age_label_1_final_preds = age_rs_model["age_SVC"].best_estimator_.predict(age_x_test_pca_85)
pd.DataFrame(age_label_1_final_preds, columns=["label_1"]).to_csv("age_label_1_final_preds.csv")

# Label 3 - Gender

In [ ]:
# Handling NaN values in the age
# Combine X and y into a single DataFrame

gender_data_train_cat = pd.concat([x_train_scaled, gender_train], axis=1)
gender_data_valid_cat = pd.concat([x_valid_scaled, gender_valid], axis=1)

# Remove rows with null values
gender_data_cleaned_train_cat = gender_data_train_cat.dropna()
gender_data_cleaned_valid_cat = gender_data_valid_cat.dropna()

# Separate X and y again
gender_x_train_cat = gender_data_cleaned_train_cat.drop(columns=["label_3"])
gender_y_train_cat = gender_data_cleaned_train_cat["label_3"]
gender_x_valid_cat = gender_data_cleaned_valid_cat.drop(columns=["label_3"])
gender_y_valid_cat = gender_data_cleaned_valid_cat["label_3"].to_frame()

In [ ]:
gender_initial = SVC()
gender_initial.fit(gender_x_train_cat, gender_y_train_cat)

gender_precision_recall_fscore_svc_initial, gender_cm_initial = validate_model(gender_initial, gender_x_valid_cat, gender_y_valid_cat)

print("Precision, Recall and F1 Score:", gender_precision_recall_fscore_svc_initial)

In [ ]:
gender_x_train_pca, gender_x_valid_pca, gender_n_components, gender_pca_cat = do_pca(
    gender_x_train_cat, gender_x_valid_cat, n_components=32
)

In [ ]:
gender_x_test_pca_85 = pd.DataFrame(
    gender_pca_cat.transform(id_x_test_cat),
    columns=["feature_pca_" + str(i) for i in range(1, gender_n_components + 1)],
)

In [ ]:
# hyperparameter tuning for svc
gender_final_models = {
    "gender_SVC": SVC(),
}

gender_grid = {
    "C": [1, 10, 100, 1000],
    "gamma": [0.1, 0.01, 0.001, 0.0001],
    "kernel": ["rbf", "poly", "sigmoid"],
    "degree": [1, 2, 3, 4, 5],
}

gender_rs_model = tune_hyperparameters(
    gender_final_models, gender_grid, gender_x_train_pca, gender_x_valid_pca, gender_y_train_cat, gender_y_valid_cat
)

gender_precision_recall_fscore_svc_final, gender_cm_final = validate_model(gender_rs_model["gender_SVC"].best_estimator_, gender_x_valid_pca, gender_y_valid_cat)


print(gender_rs_model["gender_SVC"].best_params_)

In [ ]:
gender_label_1_final_preds = age_rs_model["gender_SVC"].best_estimator_.predict(gender_x_test_pca_85)
pd.DataFrame(gender_label_1_final_preds, columns=["label_1"]).to_csv("gender_label_1_final_preds.csv")

# Label 4

In [ ]:
# Handling NaN values in the age
# Combine X and y into a single DataFrame

accent_data_train_cat = pd.concat([x_train_scaled, accent_train], axis=1)
accent_data_valid_cat = pd.concat([x_valid_scaled, accent_valid], axis=1)

# Remove rows with null values
accent_data_cleaned_train_cat = accent_data_train_cat.dropna()
accent_data_cleaned_valid_cat = accent_data_valid_cat.dropna()

# Separate X and y again
accent_x_train_cat = accent_data_cleaned_train_cat.drop(columns=["label_4"])
accent_y_train_cat = accent_data_cleaned_train_cat["label_4"]
accent_x_valid_cat = accent_data_cleaned_valid_cat.drop(columns=["label_4"])
accent_y_valid_cat = accent_data_cleaned_valid_cat["label_4"].to_frame()

In [ ]:
accent_initial = SVC()
accent_initial.fit(accent_x_train_cat, accent_y_train_cat)

In [ ]:
accent_precision_recall_fscore_svc_initial, accent_cm_initial = validate_model(accent_initial, accent_x_valid_cat, accent_y_valid_cat)

print("Precision, Recall and F1 Score:", accent_precision_recall_fscore_svc_initial)

In [ ]:
accent_x_train_pca, accent_x_valid_pca, accent_n_components, accent_pca_cat = do_pca(
    accent_x_train_cat, accent_x_valid_cat, n_components=130
)

In [ ]:
accent_x_test_pca_85 = pd.DataFrame(
    accent_pca_cat.transform(id_x_test_cat),
    columns=["feature_pca_" + str(i) for i in range(1, accent_n_components + 1)],
)

In [ ]:
# hyperparameter tuning for svc
accent_final_models = {
    "accent_SVC": SVC(),
}

accent_grid = {
    "C": [1, 10, 100, 1000],
    "gamma": [0.1, 0.01, 0.001, 0.0001],
    "kernel": ["rbf", "poly", "sigmoid"],
    "degree": [1, 2, 3, 4, 5],
}


accent_rs_model = tune_hyperparameters(
    accent_final_models, accent_grid, accent_x_train_pca, accent_x_valid_pca, accent_y_train_cat, accent_y_valid_cat
)

accent_precision_recall_fscore_svc_final, accent_cm_final = validate_model(accent_rs_model["accent_SVC"].best_estimator_, accent_x_valid_pca, accent_y_valid_cat)

print(accent_rs_model["accent_SVC"].best_params_)

In [ ]:
accent_label_1_final_preds = accent_rs_model["accent_SVC"].best_estimator_.predict(accent_x_test_pca_85)
pd.DataFrame(accent_label_1_final_preds, columns=["label_1"]).to_csv("accent_label_1_final_preds.csv")

# RESULTS